# Baselining tutorial

The aim of this tutorial is to create a baseline model. It can be later used for impact estimation when an intervention that changes the use case target value is done.

The idea behind baseline modeling is to find what whould have been the plant output if no interventions had been done. If we had two identical plants available with identical input product properties and identical operating modes, we could test the interventions in one plane and use the other one as control (baseline). However, this is extremely rare, so baseline is usually found through a model.

The model finds what would have been the target value if no interventions had been made. To do so, relevant data is selected and the model is built. Finally, the results are validated.

## Setup

In [1]:
import logging
import sys

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
# Resolve path when used in a usecase project
from pathlib import Path

sys.path.insert(0, str(Path("../../").resolve()))

First, we get our datasets. We have two datas sources available, which reperesent plant behaviour before interventions and the expected value after them.

In [3]:
from modeling import datasets

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


First we have the data before interventions

In [4]:
df_before_oai = datasets.get_sample_baselining_historical_data()
df_before_oai.describe().round(2)

,air_flow01,air_flow02,air_flow03,air_flow04,air_flow05,air_flow06,air_flow07,amina_flow,column_level01,column_level02,...,ore_pulp_flow,ore_pulp_ph,silica_conc,silica_feed,starch_flow,iron_minus_silica,feed_diff_divide_silica,total_column_level,total_air_flow,silica_conc_lagged
count,1287.00,1287.00,1287.00,1287.00,1287.00,1287.00,1287.00,1287.00,1287.00,1287.00,...,1287.00,1287.00,1287.00,1287.00,1287.00,1287.00,1287.00,1287.00,1287.00,1285.00
mean,279.50,276.14,280.25,299.05,299.13,286.31,286.56,487.28,521.74,522.09,...,397.75,9.80,12.79,14.48,2953.80,41.98,4.20,3280.99,2006.93,2.30
std,28.20,28.56,27.94,1.77,2.03,22.50,22.14,73.39,114.52,111.60,...,6.90,0.34,3.23,6.65,727.90,11.68,3.24,496.81,119.75,1.00
min,200.00,200.00,200.00,293.72,287.21,200.00,200.14,300.00,303.84,228.22,...,378.32,9.00,8.15,2.00,2000.00,15.00,0.50,2489.12,1605.37,1.00
25%,250.07,250.08,250.07,299.84,299.78,250.83,250.63,437.51,449.33,449.55,...,399.43,9.57,10.26,8.98,2256.70,33.10,1.69,2803.74,1948.68,1.53
50%,299.85,299.43,299.90,299.92,299.89,299.81,299.81,497.25,499.96,499.97,...,399.93,9.82,12.24,13.68,2935.82,42.21,3.08,3204.41,2092.12,2.02
75%,299.92,299.85,299.93,299.95,299.97,299.93,299.93,543.48,599.88,599.33,...,400.33,10.03,14.49,19.56,3462.87,50.88,5.62,3608.11,2099.09,2.87
max,300.00,300.00,300.00,300.00,300.00,300.00,300.00,698.68,800.00,800.00,...,410.00,10.77,27.77,30.00,5943.95,63.00,31.50,5094.78,2099.91,5.00


and the expected value after them. The expected value after interventions are:
* Actual sensor values for the context variables.
* Recommended controls yield by the optimizer for control variables.
* Predicted target value yield by the optimizer for the target value.

In [5]:
df_after_oai = datasets.get_sample_baselining_recs_data()
df_after_oai.describe().round(2)

,air_flow01,air_flow02,air_flow03,air_flow04,air_flow05,air_flow06,air_flow07,amina_flow,column_level01,column_level02,...,ore_pulp_flow,ore_pulp_ph,silica_conc,silica_feed,starch_flow,iron_minus_silica,feed_diff_divide_silica,total_column_level,total_air_flow,silica_conc_lagged
count,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,...,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00
mean,294.79,295.13,296.10,298.08,297.39,295.27,294.90,516.36,473.78,526.08,...,391.06,9.42,12.26,17.23,3064.88,36.61,2.57,3028.60,2071.66,2.68
std,19.85,18.85,15.26,2.02,3.73,19.47,19.48,67.17,102.48,87.61,...,11.31,0.20,3.53,5.52,627.91,8.76,1.56,431.60,93.76,1.01
min,200.00,200.00,214.62,293.46,287.82,200.06,200.00,302.13,399.21,399.06,...,378.54,9.00,7.46,7.56,2000.00,15.00,0.50,2598.05,1609.41,1.10
25%,299.82,299.71,299.89,296.54,295.02,299.94,299.71,491.44,399.82,466.39,...,381.01,9.28,9.97,13.70,2599.22,30.97,1.51,2776.49,2089.46,1.86
50%,299.88,299.84,299.92,298.71,299.87,300.00,300.00,522.26,402.65,500.13,...,386.29,9.46,11.72,16.60,2990.63,37.67,2.25,2922.25,2097.09,2.49
75%,299.93,299.93,299.94,299.92,300.00,300.00,300.00,558.30,590.60,599.50,...,400.82,9.56,13.39,20.81,3455.24,41.73,3.10,3159.40,2099.17,3.41
max,299.99,300.00,299.98,300.00,300.00,300.00,300.00,698.47,800.00,800.00,...,410.00,9.79,26.93,30.00,4563.06,52.27,6.91,4795.49,2099.71,4.95


## Data filtering

The data used to train the baseline should be historical data where no interventions have been made. To select the correct data, we perform two stepts:
1. Feature selection
2. Time period selection

### Feature selection

Baseline model features should only contain input tags that will not change because of the interventions. If an optimization step is performed, the following tags have to be avoided:
- Tags that are used for optimization should be excluded. Otherwise, a change cause by the interventions would change the baseline value and it would not be useful for impact calculation.
- Output tags could be equally modified by the interventions. Hence, they should be excluded.

In [6]:
model_features = ['iron_feed', 'silica_feed', 'feed_diff_divide_silica']
model_features

['iron_feed', 'silica_feed', 'feed_diff_divide_silica']

We also define the target value and the datetime column.

In [7]:
target_column = "silica_conc"
datetime_column = "timestamp"

### Time period selection

Time periods should...
- not overlap with any other initiative to have a clean impact narrative
- represent the maximum number of possible plant conditions, such as
    - operating modes
    - type and quality of input materials
    - seasonality
    - equipment shutdowns
    - ...
 
In this example we will assume that the data we have available meets this criteria. In a real scenario, this should be checked with client and SMEs.

## Model training

We will build the baseline model using the functionalities of the `modeling` package. We assume familiarity with them for this tutorial. Please check the [modeling tutorial](./modeling.ipynb) to learn how to use them in more detail.

First, we split the data in train and test sets. To learn more about the splitting options, check the [splitter tutorial](./splitter_base.ipynb).

In [8]:
from modeling import create_splitter, split_data
splitter = create_splitter(
    "last_window", 
    splitting_parameters={
        "datetime_column": datetime_column,
        "freq": "14D",
    },
)
train_data, test_data = split_data(df_before_oai, splitter)

INFO:modeling.splitters._splitters.base_splitter:Length of data before splitting is 1287
INFO:modeling.splitters._splitters.by_last_window:Splitting by datetime: 2017-08-16 20:00:00
INFO:modeling.splitters._splitters.base_splitter:Length of the train data after splitting is 1175, length of the test data after splitting is 112.


After this step, we need to choose which model class we will be using for training. Here, we will explore the main two approaches:
- Linear models
- KNN

**We recommend to use linear models whenever possible**. The linear model is not as popular due to lack of usage and explanation guide. For more details of comparison see the brix post "OAI impact calculation guidelines".

### Linear model

#### Pipeline inicialization

First we initialize our model. To do so, we use the same set up as in training any other models. For details on how to set up a model training pipeline see the [modeling tutorial notebook](./modeling.ipynb).

Below we set up a linear model. Instead of using a simple linear regression, we will use a `ElasticNet` model to be more flexible in choosing hyperparameters. We also add a `StandardScaler` to standardize the data before it.

In [9]:
from modeling import create_model_factory


linear_pipeline_init_config = {
    'estimator': {
        'class_name': 'sklearn.linear_model.ElasticNet',
        'kwargs': {
            'random_state': 123,
            'max_iter': 1000
        }
    },
    'transformers': [
        {
            'class_name': 'sklearn.preprocessing.StandardScaler',
            'kwargs': {},
            'name': 'standard_scaler',
            'wrapper': 'preserve_columns',
        }
    ]
}

linear_pipeline_factory = create_model_factory(
    model_factory_type="modeling.SklearnPipelineFactory",
    model_init_config=linear_pipeline_init_config,
    features=model_features,
    target=target_column,
)

linear_pipeline = linear_pipeline_factory.create()
linear_pipeline

SklearnPipeline(estimator=Pipeline(steps=[('standard_scaler',
                 SklearnTransform(transformer=StandardScaler())),
                ('estimator', ElasticNet(random_state=123))]), target="silica_conc" ,features_in=['iron_feed', 'silica_feed', 'feed_diff_divide_silica'], features_out=None)

#### Hyperparameter tuning

Next, we will tune our model hyperparameters. Again, more detail on the tuning process can be found on the [modeling tutorial notebook](./modeling.ipynb).

In [10]:
from modeling import create_tuner, tune_model

linear_pipeline_tuner_config = {
    'class_name': 'sklearn.model_selection.GridSearchCV',
    'kwargs': {
        'n_jobs': -1,
        'refit': 'mae',
        'param_grid': {
            'estimator__alpha': [0.1, 0.3, 0.5, 0.7, 0.9, 1],
            'estimator__l1_ratio': [0.01, 0.2, 0.4, 0.6, 0.8, 1],
        },
        'scoring': {
            'mae': 'neg_mean_absolute_error',
            'rmse': 'neg_root_mean_squared_error',
            'r2': 'r2',
        },
        'cv': 5,
    }
}

linear_pipeline_tuner = create_tuner(
    model_factory=linear_pipeline_factory,
    model_tuner_type="modeling.SklearnPipelineTuner",
    tuner_config=linear_pipeline_tuner_config,
)

linear_pipeline = tune_model(
    model_tuner=linear_pipeline_tuner,
    data=train_data,
    hyperparameters_config=None,
)

print("The best params:\n")
for k, v in linear_pipeline.estimator.get_params().items():
    print(f"{k}: {v}")

INFO:modeling.models.sklearn_pipeline.model:`features_out` attribute is not specified. Setting `features_out` based on factual data.
INFO:modeling.models.sklearn_pipeline.tuner:Initializing sklearn hyperparameters tuner...
INFO:modeling.models.sklearn_pipeline.tuner:Tuning hyperparameters...
The best params:

alpha: 0.1
copy_X: True
fit_intercept: True
l1_ratio: 0.01
max_iter: 1000
positive: False
precompute: False
random_state: 123
selection: cyclic
tol: 0.0001
warm_start: False


We will use the hyperparameters shown above to train the model. We osberve that the best parameters prioritize l2 regularization in contrast with l1 regulatization.

#### Model training

Finally, we can train the model using the pipeline after hyperparameter tuning.

In [11]:
from modeling import train_model

linear_pipeline = train_model(linear_pipeline, train_data)

#### Model evaluation

We will show main metrics and feature importance. Feature importance should be checked with SMEs to ensure that it correctly represents the expected behaviour.

In [12]:
from modeling import calculate_metrics
import pandas as pd

train_metrics = calculate_metrics(
    train_data, model=linear_pipeline,
)

test_metrics = calculate_metrics(
    test_data, model=linear_pipeline,
)

linear_metrics = pd.DataFrame(
    {
        "train_metric_value": train_metrics,
        "test_metric_value": test_metrics,
    },
).rename_axis("metric_name")

linear_metrics

,train_metric_value,test_metric_value
metric_name,,
mae,2.405719,2.413806
rmse,3.177314,2.999873
mse,10.095326,8.999240
mape,0.189729,0.196221
r_squared,0.046954,-0.052942
var_score,0.046954,-0.052674


In [13]:
default_importance = linear_pipeline.get_feature_importance(
    train_data,
)
shap_importance = linear_pipeline.get_shap_feature_importance(
    train_data[linear_pipeline.features_in],
)
linear_importance_table = pd.DataFrame(
    {
        "default_importance": default_importance,
        "shap_importance": shap_importance,
    }
)
linear_importance_table

INFO:modeling.models.sklearn_pipeline.model:Estimator of type <class 'sklearn.linear_model._coordinate_descent.ElasticNet'> does not have `feature_importances_` using sklearn.inspection.permutation_importances instead.
INFO:modeling.models.sklearn_pipeline.model:`Using model-agnostic` <class 'shap.explainers._exact.ExactExplainer'>` to extract SHAP values... `shap` can't apply model-specific algorithms for <class 'modeling.models.sklearn_pipeline.model.SklearnPipeline'>. Consider switching to `SklearnModel` if computation time or quality don't fit your needs.


ExactExplainer explainer: 1176it [00:13, 36.47it/s]                                                                     


,default_importance,shap_importance
iron_feed,-0.000075,0.011944
silica_feed,0.049934,0.427674
feed_diff_divide_silica,0.187737,0.786483


### Nearest neighbor regressor

Here we will repeat the procedure using a KNN model.

#### Pipeline inicialization

In [14]:
knn_pipeline_init_config = {
    'estimator': {
        'class_name': 'sklearn.neighbors.KNeighborsRegressor',
        'kwargs': {
        }
    },
    'transformers': [
        {
            'class_name': 'sklearn.preprocessing.StandardScaler',
            'kwargs': {},
            'name': 'standard_scaler',
            'wrapper': 'preserve_columns',
        }
    ]
}

knn_pipeline_factory = create_model_factory(
    model_factory_type="modeling.SklearnPipelineFactory",
    model_init_config=knn_pipeline_init_config,
    features=model_features,
    target=target_column,
)

knn_pipeline = knn_pipeline_factory.create()
knn_pipeline

SklearnPipeline(estimator=Pipeline(steps=[('standard_scaler',
                 SklearnTransform(transformer=StandardScaler())),
                ('estimator', KNeighborsRegressor())]), target="silica_conc" ,features_in=['iron_feed', 'silica_feed', 'feed_diff_divide_silica'], features_out=None)

#### Hyperparameter tuning

In [15]:
knn_pipeline_tuner_config = {
    'class_name': 'sklearn.model_selection.GridSearchCV',
    'kwargs': {
        'n_jobs': -1,
        'refit': 'mae',
        'param_grid': {
            'estimator__weights': ['uniform', 'distance'],
            'estimator__algorithm': ['ball_tree', 'kd_tree', 'brute'],
            'estimator__n_neighbors': [10, 20, 30, 40],
        },
        'scoring': {
            'mae': 'neg_mean_absolute_error',
            'rmse': 'neg_root_mean_squared_error',
            'r2': 'r2',
        },
        'cv': 5,
    }
}

knn_pipeline_tuner = create_tuner(
    model_factory=knn_pipeline_factory,
    model_tuner_type="modeling.SklearnPipelineTuner",
    tuner_config=knn_pipeline_tuner_config,
)

knn_pipeline = tune_model(
    model_tuner=knn_pipeline_tuner,
    data=train_data,
    hyperparameters_config=None,
)

print("The best params:\n")
for k, v in knn_pipeline.estimator.get_params().items():
    print(f"{k}: {v}")

INFO:modeling.models.sklearn_pipeline.model:`features_out` attribute is not specified. Setting `features_out` based on factual data.
INFO:modeling.models.sklearn_pipeline.tuner:Initializing sklearn hyperparameters tuner...
INFO:modeling.models.sklearn_pipeline.tuner:Tuning hyperparameters...
The best params:

algorithm: kd_tree
leaf_size: 30
metric: minkowski
metric_params: None
n_jobs: None
n_neighbors: 30
p: 2
weights: distance


#### Model training

In [16]:
knn_pipeline = train_model(knn_pipeline, train_data)

#### Model evaluation

In [17]:
train_metrics = calculate_metrics(
    train_data, model=knn_pipeline,
)

test_metrics = calculate_metrics(
    test_data, model=knn_pipeline,
)

knn_metrics = pd.DataFrame(
    {
        "train_metric_value": train_metrics,
        "test_metric_value": test_metrics,
    },
).rename_axis("metric_name")

knn_metrics

,train_metric_value,test_metric_value
metric_name,,
mae,1.243789,2.829170
rmse,2.130970,3.566096
mse,4.541032,12.717039
mape,0.098479,0.230056
r_squared,0.571305,-0.487937
var_score,0.571997,-0.484464


In [18]:
default_importance = knn_pipeline.get_feature_importance(
    train_data,
)
shap_importance = knn_pipeline.get_shap_feature_importance(
    train_data[knn_pipeline.features_in],
)
knn_importance_table = pd.DataFrame(
    {
        "default_importance": default_importance,
        "shap_importance": shap_importance,
    }
)
knn_importance_table

INFO:modeling.models.sklearn_pipeline.model:Estimator of type <class 'sklearn.neighbors._regression.KNeighborsRegressor'> does not have `feature_importances_` using sklearn.inspection.permutation_importances instead.
INFO:modeling.models.sklearn_pipeline.model:`Using model-agnostic` <class 'shap.explainers._exact.ExactExplainer'>` to extract SHAP values... `shap` can't apply model-specific algorithms for <class 'modeling.models.sklearn_pipeline.model.SklearnPipeline'>. Consider switching to `SklearnModel` if computation time or quality don't fit your needs.


ExactExplainer explainer: 1176it [00:17, 28.43it/s]                                                                     


,default_importance,shap_importance
iron_feed,0.640723,0.662018
silica_feed,0.607175,0.564240
feed_diff_divide_silica,0.638325,0.653207


## Model validation

We will showcase the validation approach with the linear model. As it is the preferred approach for baselining and its metrics on the test set are better than the KNN approach, it is better baseline candidate. To do so, we need to perform two checks:

* Model errors have an average of 0.
* Uplifts found through interventions are not caused by the model error.

Checks need to be performed over a subset of data that has not been used for training, to mimic the live behaviour of the model. It also needs to be a historical period where no interventions have been made. Because of that, the errors of the test set are a good candidate for this.

In [19]:
from modeling import calculate_model_predictions

linear_test_predictions = calculate_model_predictions(
    test_data, linear_pipeline,
)
model_errors = test_data[target_column] - linear_test_predictions["model_prediction"]

### Average model error is 0

To check whether the average model error is consistent with 0, we will perform hypothesis testing with the following hypothesis:

* H0: Model errors have an average of 0.
* H1: Model errors have an average different than 0.

Here, we will assume that the model error distribution is such that a T-test can be applied. This is a reasonable assumption every time that the sample size is greater than 30. If it is not the case, other techniquest to perform hypothesis testing, such as bootstrapping, can be explored.

In [20]:
model_errors.shape[0]

112

In [21]:
from scipy.stats import ttest_1samp

ttest_1samp(model_errors, popmean=0, alternative='two-sided')

TtestResult(statistic=0.16782919502110405, pvalue=0.8670230527053208, df=111)

P-value is not big enough, so we do not have the evidence to reject the null hypothesis (model errors are 0).

### Minimum intervention average value

Now, we wish to check if we are able to capture the expected uplift from our interventions using this baseline model.

To do so, we perform a hypothesis test with the following hypothesis:

* H0: Uplift and model error have the same mean.
* H1: Uplift has a greater (or smaller, depending on the use case) mean than the model error.

We will assume that the uplifts mean is expected to be smaller than the model error. Again, we will use a T-test because our sample size is big enough. 

In [22]:
df_after_oai.shape[0]

81

In [23]:
after_oai_predictions = calculate_model_predictions(
    df_after_oai, linear_pipeline,
)
uplift = df_after_oai[target_column] - after_oai_predictions["model_prediction"]
uplift.mean()

-0.8041783222535256

In [24]:
from scipy.stats import ttest_ind

ttest_ind(uplift, model_errors, equal_var=False, alternative='less')

TtestResult(statistic=-1.7525977362331289, pvalue=0.040824768181986215, df=154.90741383442028)

As the p-value is smaller than 0.05, which is the usuall threshold to reject the null hypothesys, we reject that the uplifts and the model errors have the same mean in favour of the uplifts having a smaller mean than the model errors.